<img src="https://github.com/rvhonorato/workflow_template/blob/main/workflow_template/images/logos.png?raw=1">



In [ ]:
# Install miniconda
%%capture
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX 
rm Miniconda3-4.5.4-Linux-x86_64.sh

In [3]:
!git clone https://github.com/rvhonorato/workflow_template.git
%cd workflow_template

Cloning into 'workflow_template'...
remote: Enumerating objects: 354, done.
remote: Counting objects: 100% (354/354), done.
remote: Compressing objects: 100% (279/279), done.
remote: Total 354 (delta 75), reused 330 (delta 67), pack-reused 0
Receiving objects: 100% (354/354), 9.38 MiB | 15.23 MiB/s, done.
Resolving deltas: 100% (75/75), done.
/content/workflow_template


In [7]:
# Install required packages,
#  this step might take a few minutes, go get a ☕
!conda env create -f workflow_template/environment.yml

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

In [10]:
# Initialize this notebook's style and functionalities
%run src/init_notebooks.py
#hide_toggle()
# %cd workflow_template
# from IPython.display import HTML
# html = HTML(open("src/gromacs-training.css").read())
# display(html)
# Setup complete! :)

# A GROMACS - HADDOCK antibody workflow 

```
Author                : Alessandra Villa, Zuzana Jandova
Goal                  : Jupyter notebook for training purposes in antibody simulation and docking
Time                  : 10 minutes reading time, no computation wait time
Prerequisites         : Know how to run an md simulation, register with HADDOCK
Software requirements : GROMACS (version 2020) , pdb-tools, python 2.7, HMMER, Biopandas, Biopython
Tested for            : 
```
 

# Table of Contents

* [Use Case Introduction](#UCintro)
* [Description of the Workflow](#Workflow)
* [Structure preparation](#Structure)
* [Molecular Dynamics with GROMACS](#MD)
* [CDRs identification](#CDRs)
* [Cluster antibody trajectory](#Cluster) 
* [Prepare HADDOCK json file](#HADDOCK)

 ## <a class="anchor" id="UCintro" > Use Case Introduction </a>


Antibody design has grown into one of the fastest growing branches in the pharmaceutical industry. Antibodies promise extremely high specificity and the ability to use the body’s own immune system to kill e.g. tumors, however  their size and complexity make their computational design challenging. 
 
An antibody is a large protein that generally works by attaching itself to an antigen, which is a unique site of the pathogen. The binding harnesses the immune system to directly attack and destroy the pathogen. Antibodies can be highly specific while showing low immunogenicity, which is achieved by their unique structure. **The fragment crystallizable region (Fc region**) activates the immune response and is species specific, i.e. human Fc region should not evoke an immune response in humans.  **The fragment antigen-binding region (Fab region**) needs to be highly variable to be able to bind to antigens of various nature (high specificity).  The terminal (antigen recognising) domain of the Fab region is caleld **the variable domain (Fv domain)**.
 

<figure >
<img src="https://github.com/rvhonorato/workflow_template/blob/main/workflow_template/images/antibody_described.png?raw=1">
</figure>

The small part of the Fab region that binds the antigen is called **paratope**. The part of the antigen that binds to an antibody is called **epitope**. The paratope consists of six highly flexible loops, known as **complementarity-determining regions (CDRs)** or hypervariable loops whose sequence and conformation are altered to bind to different antigens.  


 - Specific problem 

 - Introduction of use case as tutorial
 what they will learn from a tutorial
  
 - References
     - reading material


 ## <a class="anchor" id="Workflow"> Description of the Workflow </a>

This jupyter notebook combines two approaches: molecular dynamics (MD) simulation with [GROMACS](http://www.gromacs.org/About_Gromacs) and molecular docking with [HADDOCK](https://www.bonvinlab.org/software/haddock2.4/) to provide a good starting point for antibody design. We improve the sampling of the CDRs  using MD, extract the most diverse loop conformations and prepare such ensemble for running with HADDOCK. 



<img src="https://github.com/rvhonorato/workflow_template/blob/main/workflow_template/images/UC1_example.png?raw=1" width="500">


To obtain the best prediction of a bound antibody-antigen complex, we will be using a worflow consisting of several steps. 
<img src="https://github.com/rvhonorato/workflow_template/blob/main/workflow_template/images/workflow.png?raw=1" width="900">


1. Download PDBs of antibody and antigen
1. Pre-processing antibody pdb for HADDOCK
1. Converting antibody pdb into GROMACS
1. Generation of mdrun input file  
1. Setup MD of unbound antibody (gmx) run mdrun (run local or on HPC - script)
1. Trajectory postprocessing (remove pbc) gmx trjconv 
1. Define loop residues – (protocol Ambrosetti et al., 2020)
1. Index on loop residues & backbone (gmx)
1. Cluster MD of unbound antibody by loop residue conformations (gmx cluster) gromos
1. extract 20 most populated cluster (automatic with gromos)
1. Prepare clusters from MD for docking (renumber) ⟹ setup docking run json file






 ## System setting

The starting point is a antibody structure file. For this tutorial, we will utilize a Fab part of an antibody (PDB code [3RVT](http://www.rcsb.org/structure/3RVT)) which binds to the group 1 house dust mite allergen (PDB code [3F5B](http://www.rcsb.org/structure/3F5B)). As a reference, a crystal structure of the complex is available (PDB code [3RVW](http://www.rcsb.org/structure/3RVW)). All files are available from the RCSB website, https://www.rcsb.org/. For this tutorial, the PDB file for the crystal structure is depositied in `input` directory as "3RVT.pdb" .

Below you can visualize the antibody structure  

In alternative you can visualize the structure using a viewing program such as VMD.
Note: close the VMD window after you are done looking at the protein to continue with this notebook

# <a class="anchor" id="Structure">Structure preparation </a>

## Get your pdb

In [ ]:
# Install ANARCI
cd src/anarci-1.3
python2.7 setup.py install
cd ../..

In this step we will make use of the local version of [PDB-tools](http://www.bonvinlab.org/pdb-tools/). PDB-tools were designed to be a swiss-knife for the PDB format. They have no external dependencies, besides the Python programming language. You can find them on [Github](https://github.com/haddocking/pdb-tools) or as a webserver. [PDB-tools webserver](https://wenmr.science.uu.nl/pdbtools/) is a powerful tool that enables you to edit pdbs quickly and painlessly without any scripting knowledge.

In [ ]:
%cd data

In [ ]:
!pdb_fetch -biounit 3RVT > 3RVT.pdb

In [ ]:
representations=[
    {"type": "cartoon", "params": {
        "sele": "protein and not ANI", "color": "chainname", 
    }},
    {"type": "ball+stick", "params": {
        "sele": "hetero"
    }},]   

In [ ]:
import nglview as ng
#view = ng.show_structure_file("3RVT.pdb")
view = ng.show_structure_file("3RVT.pdb", defaultRepresentation= False)
view.representations = representations
view
# click and drag to rotate, zoom with your mouseweel 
# for more infor on this viewer have a look at https://github.com/nglviewer/nglview

### Clean your pdb


In [ ]:
!pdb_chainxseg 3RVT.pdb > 3RVT_seg.pdb 
!pdb_splitseg 3RVT_seg.pdb 
!pdb_reres -501 3RVT_seg_D.pdb  > 3RVT_seg_D_ren.pdb  
!pdb_merge 3RVT_seg_C.pdb 3RVT_seg_D_ren.pdb > 3RVT_merged.pdb 
!pdb_chain -A 3RVT_merged.pdb | pdb_seg| pdb_delhetatm | pdb_tidy  > 3RVT_clean.pdb  
!sed -i "" '/ANISOU/d'  3RVT_clean.pdb  >> 3RVT_clean.pdb
!rm *merged.pdb *seg* 

In [ ]:
%ls 3RVT_clean.pdb
view = ng.show_file("3RVT_clean.pdb",  defaultRepresentation= False)
view.representations = representations
view

Once you've had a look at the molecule, you are going to check that only the protein is present in the pdb file. Otherwise strip out all the other molecules in the crystal . To delete the other molecules , either use a plain text editor like vi, emacs (Linux/Mac), or Notepad (Windows). Do not use word processing software! 

Always check your .pdb file for entries listed under the comment MISSING, as these entries indicate either atoms or whole residues that are not present in the crystal structure. Terminal regions may be absent, and may not present a problem for dynamics.

#  <a class="anchor" id="MD"> Molecular Dynamics with GROMACS </a >

## Generating a topology

Now the PDB file should contain only protein atoms, and is ready to be input into GROMACS. 
The first GROMACS tool, we use, is pdb2gmx. The purpose of pdb2gmx is to generate three files:

* The topology for the molecule.
* A position restraint file.
* A post-processed structure file. 

The topology (topol.top by default) contains all the information necessary to define the molecule within a simulation. This information includes nonbonded parameters (atom types and charges) as well as bonded parameters (bonds, angles, dihedrals and atom connectivity).

## Force Field

Here, we made an important decision for the course of the simualtion in choosing the force field. Here we use CHARMM36m all-atom force field. Check [here](http://mackerell.umaryland.edu/charmm_ff.shtml#gromacs) for update in the CHARMM36 force field implementation for GROMACS . The force field files contain the information that will be written to the topology. 

In [ ]:
!tar xvf input/charmm36.tar

Here we excecute [gmx pdb2gmx](http://manual.gromacs.org/documentation/current/onlinehelp/gmx-pdb2gmx.html)

In [ ]:
!gmx pdb2gmx -f input/3RVW_antibody.pdb -p antibody.top -o antibody.gro -i posre -ff charmm36-jul2017 -water tip3p -ignh

## Defining a simulation box

In [ ]:
!gmx editconf -f antibody.gro -d 0.7 -bt dodecahedron -o antibody_box.gro

## Solvating the simulation system

In [ ]:
!gmx solvate -cp antibody_box.gro -p antibody.top -o water.gro

## Adding ions 

We now have a solvated system that contains a charged protein. The output of pdb2gmx told us that the protein has a net charge of 3e (based on its amino acid composition). If you missed this information in the pdb2gmx output, look at the last line of each [ atoms ] directive in topology file; it should read  "qtot -1." for chain A and  "qtot 4." for chain B. Since life does not exist at a net charge, we must add ions to our system. Further, we aim to approximate physiological conditions and use therefore a NaCl concentration of 0.15 M.

In [ ]:
!touch ion.mdp
!gmx grompp -f ion.mdp -c water.gro -p antibody.top -o
!echo 13 | gmx genion -s topol.tpr -p antibody.top -neutral -conc 0.15 -o antibody.gro

# Energy minimization

The solvated, electroneutral system is now assembled. Before we can begin dynamics, we must ensure that the system has no steric clashes or inappropriate geometry. The structure is relaxed through a process called energy minimization (EM).

In [ ]:
 !cat input/emin-charmm.mdp

In [ ]:
!gmx grompp -f input/emin-charmm.mdp -c antibody.gro -p antibody.top -o em.tpr 
!gmx mdrun -v -s em.tpr -deffnm antibody_em

#gmx grompp -f ../../../mdp_files/md_charmm36m.mdp -c POS/confout.gro -p $name.top -o MD/topol.tpr
#cp topol.tpr in topol100.tpr
#gmx convert-tpr -s topol100.tpr -extend 400000 -o topol.tpr

# Relaxing solvent and ions positions

EM ensured that we have a reasonable starting structure, in terms of geometry and solvent orientation. To begin real dynamics, we must equilibrate the solvent and ions around the protein. If we were to attempt unrestrained dynamics at this point, the system may collapse. The reason is that the solvent is mostly optimized within itself, and not necessarily with the solute, and ions are randomly placed by replacing water molecules.

Among other files, gmx pdb2gmx generated a file, called posre.itp  The purpose of posre.itp is to apply a position restraining force on the heavy atoms of the protein (anything that is not a hydrogen). Movement is permitted, but only after overcoming a substantial energy penalty. The utility of position restraints is that they allow us to relax our solvent and ions around our protein, without the added variable of structural changes in the protein. The origin of the position restraints (the coordinates at which the restraint potential is zero) is provided via a coordinate file passed to the -r option of grompp. Depending from the protein and ion types, this process may also be in the order nanoseconds.

In [ ]:
!cat input/md_eq_posre_charmm36m.mdp

In [ ]:
!gmx grompp -f input/md_eq_posre_charmm36m.mdp -c antibody_em.gro -p antibody.top -o posre.tpr -r antibody_em.gro
!gmx mdrun -v -s posre.tpr -deffnm antibody_posre

Upon completion of the the equilibration phase, the system is now equilibrated at the desired temperature and pressure. We are now ready to release the position restraints and run production MD for data collection. We will make use of the checkpoint file (which in this case now contains preserve pressure coupling information) to grompp. We will run a 100ns MD simulation. We use velocity-rescaling temperature coupling as thermostat and stochastic cell rescaling as barostat. A full explanation of the available thermostats and barostats in GROMACS can be found in the manual (see [here](http://manual.gromacs.org/documentation/current/reference-manual/algorithms/molecular-dynamics.html#temperature-coupling) for thermostat and [here](http://manual.gromacs.org/documentation/current/reference-manual/algorithms/molecular-dynamics.html#pressure-coupling) for barostat)

In [ ]:
!cat input/md_charmm36m.mdp

In [ ]:
!gmx grompp -f input/md_charmm36m.mdp -c antibody_posre.gro -t antibody_posre.cpt -p antibody.top -o antibody_md.tpr
!gmx mdrun -ntmpi 1 -v -deffnm antibody_md

Approach to select the structure for HADDOCK 
* fit backbone
* selection of variable loop (Ambrosetti et al.2020)
* cluster analysisi
* 20 high populated clusters will be given as input to HADDOCK 

In [ ]:
!echo 1 | gmx trjconv -f antibody_md.xtc -s em.tpr -pbc nojump -o traj_nojump.xtc

In [ ]:
!echo 4 1 | gmx trjconv -f traj_nojump.xtc -s em.tpr -fit rot+trans -o antibody_fit.xtc

In [ ]:
import nglview as ng 
import mdtraj as md
traj = md.load("antibody_fit.xtc",top="antibody_box.gro")
view = ng.show_mdtraj(traj)
view

In [ ]:
!echo 20 1 |gmx cluster -h -f antibody_fit.xtc -cl cluster-gromos.pdb -s em.tpr -n input/loop_index.ndx -b 1000 -method gromos -cutoff 0.13 -nofit -g cluster-gromos.log

In [ ]:
!echo 20 1 |gmx cluster -f antibody_fir.xtc -cl cluster-jp.pdb -s em.tpr -n input/loop_index.ndx -b 1000 -method jarvis-patrick -cutoff 0.13 -nofit -g cluster-jp.log

# <a class="anchor" id="CDRs"> CDRs identification </a >

Antibodies are created by two identical pairs of  **light (L)** and **heavy (H)** chains. As shown above the variable domains of both chains are involved in the antigen recognition [1]. Each chain contains three <font color="#8F0306">**CDRs** (in red) </font>in their variable domain, which show the highest level of variability and directly interact with the antigen [2, 3]. 
Each CDR contains one <font color="#8F0306">**hyper variable loop (HV)** </font>(six loops in total) which are crucial for the recognition of the cognate antigen.

<img src="https://github.com/rvhonorato/workflow_template/blob/main/workflow_template/images/CDRs.png?raw=1" width="600">

In this notbeook we will be using the protocol of [Ambrosetti, et al ArXiv, 2020](https://www.biorxiv.org/content/10.1101/2020.03.18.967828v1) to identify CDRs residues and convert them to the GROMACS format for further trajectory clustering.

1. Novotný J, Bruccoleri R, Newell J, et al (1983) Molecular anatomy of the antibody
binding site. J Biol Chem 258:14433–14437
2. Sela-Culang I, Kunik V, Ofran Y (2013) The Structural Basis of Antibody-Antigen
Recognition. Front Immunol 4:302. https://doi.org/10.3389/fimmu.2013.00302
3. MacCallum RM, Martin ACR, Thornton JM (1996) Antibody-antigen interactions:
Contact analysis and binding site topography. J Mol Biol 262:732–745.
https://doi.org/10.1006/jmbi.1996.0548

In [ ]:

%cd data
# Renumber antibody with the Chothia scheme
!python2.7 ../scripts/ImmunoPDB.py -i input/3RVT.pdb -o 3RVT_ch.pdb --scheme c --rename --splitscfv

# Format the antib-ody in order to fit the HADDOCK format requirements
# and extract the HV loop residues and save them into a file
!python ../scripts/ab_haddock_format.py 3RVT_ch.pdb 3RVT_HADDOCK.pdb A > active.txt

# Add END and TER statements to the .pdb file
!pdb_tidy 3RVT_HADDOCK.pdb > oo; mv oo 3RVT_HADDOCK.pdb


In [ ]:
!pwd
%cd ../

## Visualise CDR residues

In [ ]:
f = open("data/active.txt")
lines=f.read()
representations=[
    {"type": "cartoon", "params": {
        "sele": "protein and not ANI", "color": "chainname", 
    }},
    {"type": "hyperball", 'params': {"sele":lines, "color":"orange"}}
]        

In [ ]:


print(lines)
view = ng.show_file('data/3RVT_HADDOCK.pdb', defaultRepresentation= False)
view.representations = representations
view

## Turn CDR residues into index file to cluster antibody trajectory

In [ ]:
%cd data 

!echo -e  $(sed -e  '1 s/./ri /' -e 's/,/ /g'  active.txt)'\n' name 15 CDRs '\n'q  | gmx make_ndx -f topol.tpr -o index_jupy.ndx

In [ ]:
!tail -n 45 index_jupy.ndx

# <a class="anchor" id="Cluster">  Cluster antibody trajectory </a>

## Create an ensemble for docking

In [ ]:
%cd data
!pdb_mkensemble input/3RVT.pdb input1.pdb input1.pdb > 3RVT_ensemble.pdb

# <a class="anchor" id="HADDOCK">  Prepare HADDOCK `json` file </a>

We prepared a sample HADDOCK `json` file that one can submit to the [Submit file interface](https://bianca.science.uu.nl/haddock2.4/submit_file) of HADDOCK2.4. First one needs to register  for all HADDOCK services [https://bianca.science.uu.nl/auth/register/](https://bianca.science.uu.nl/auth/register/). Then we will modify the `json` file with a few available scripts to update it with our newly created pdb. In this case we use ambiguous restraints created from the reference crystal structure (PDB ID [3RVW](https://www.rcsb.org/structure/3RVW)). They were defined based on the true interface (all residues within 3.9Å from the other protein) and are also located in `data/input/ambig.tbl`.  If one wishes to replace them too, there is a script `data/script/replace_tbl.py` which updates the restraint as well. Here one needs to specify which type of restraints they want to replace (`-type (ambig,unambig,hbond,dihedral)`). In this scenario we are using increased sampling such that each antibody conformer is used in 100 model for it0 - rigid body docking. In this case we have an ensemble of 12 conformers, thus 1200 structures geenrated in it0 (`structures_0`). Further, we increase sampling from 200 to 400 per it1 (`structures_1`) and it1 water (`waterrefine`) and in the analysis part (`anastruc_1`) too. These changes are already included in the sample `json` file, thus do not need to be modified manually.   


### Modified sampling parameters:

`"anastruc_1": 400,
"structures_0": 1200,
"structures_1": 400,
"waterrefine": 400,`

In [ ]:
%cd data
python  ../scripts/rswieplace_pdb.py  -param input/job_params.json -pdb input/3RVT_ensemble.pdb -i 1 > new.json

Visit [https://bianca.science.uu.nl/haddock2.4/submit_file](https://bianca.science.uu.nl/haddock2.4/submit_file) and upload the `json` file. 

In [ ]:
%pwd

# Writing the notebook

- Style the notebook with style sheet provided in src
    - Rationale: Easy way to set context for notebooks, create a "GROMACS"-tutorial feel
    - Con: requires users to execute the first cell
    - Pro: can serve as an introduction on how to execute cells

- Write headers (# title, ## header2, etc.) in seperate cells 
    - Rationale: this enables folding of sections 

- Include images as

![ImageNotFoundAlternativeTxt](images/Bioexcel_symbol.svg "Text you will see when hovering over the logo with the mouse")

![ImageNotFoundAlternativeTxt](images/non-existant.svg "Logo Title Text 1")


- Toggle solutions with the hide_toggle() function, hide the next cell with for_next=True

In [ ]:
#hide this cell
hide_toggle()

In [ ]:
hide_toggle(for_next=True)

In [ ]:
#some solution that should not yet be visible

more text

# Common pitfalls

- Cells are stuck in evaluation (marked with `[*]`)
    - Reason: Cells are evaluated serially. Jupyter cannot run bash commands in the background. Especially when opening another programm, like vmd, the notebook requires that the window is closed before going further
    - Solution : Make workshop participants aware of this 
    - Solution : Show users the kernel -> interrupt solution
    - Solution : start evaluations in subprocess
        - Con : spawning subprocesses from within notebooks depends on the python verion. 
        - Con : This requires some python boilerplate code that makes it harder to have a one-to-one correspondence of the command line command and what users will read in the notebook
- Users don't notice that they can execute / run input cells with the run botton, but rather copy-paste or type into the command line
- Users feel like they are taken through the notebooks via "auto-pilot"
    - Solution: Include the `hide_toggles()` function
    - Solution: Use quizzes
    - Con: This requires executing the first line of code in the notebook to work well
- The notebook is unaware of the changes to shell environment and variables changes when executed with '! command!
    - Reason: Bash commands are executed in subshells 
- '!cd directory' does not work as expected
    - Reason: Bash commands are executed in subshells 
    - Solution: use `%cd ` or `cd ` to permanently change the current directory
    - Con: this is notebook 'magic' that might confuse useres
- Loading environment modules does not work
    - Reason: Bash commands are executed in subshells
    - Solution: Use python module commands
       - Con : Depends on the python version and importing external modules
- Markdown cell line breaks look differnetly when markdown cells are executed
- Showing contents of long files requires lots of scrolling
  - Solution: use `%less filename`
- Longer bash script is hard to read in a cell
  - Solution: use `%bash` on top of the cell, then continue with usual bash